 Login to your azure account az login - device

 Ensure you change the kernel to Python 3.10 AzureML

In [ ]:
pip install python-dotenv

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

In [4]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from local.env file
load_dotenv('local.env')

In [ ]:
# Get Azure ML credentials from environment variables
subscription_id = os.getenv('AZUREML_SUBSCRIPTION_ID')
resource_group = os.getenv('AZUREML_RESOURCE_GROUP')
workspace = os.getenv('AZUREML_WS_NAME')

# Print values for debugging (remove in production)
print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Workspace: {workspace}")

# Create ML client with the credentials
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [ ]:
file_model = Model(
    path="models/phi-4-mini/onnx",
    type=AssetTypes.CUSTOM_MODEL,
    name="fine-tuning-phi-4-mini-onnx-int4-cpu",
    description="Fine tuning by MSOlive",
)

In [ ]:
registered_model = ml_client.models.create_or_update(file_model)

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/a100compute/code/Users/lokinfey/workshop/models/phi-4-mini/onnx' 'https://slmworkshop2695423610.blob.core.windows.net/azureml-blobstore-4f739eb1-140b-41a8-8084-8620be2bb1a0/LocalUpload/7740bb718f31d0f7a1fdda75912ad291/onnx' 

See https://learn.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading onnx (4975.45 MBs): 100%|██████████| 4975451725/4975451725 [00:11<00:00, 428081869.68it/s]




Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'fine-tuning-phi-4-mini-onnx-int4-cpu', 'description': 'Fine tuning by MSOlive', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/901e11c6-42ed-4fcc-875a-6bd77390af5b/resourceGroups/AzureAIGroup/providers/Microsoft.MachineLearningServices/workspaces/slm-workshop/models/fine-tuning-phi-4-mini-onnx-int4-cpu/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/a100compute/code/Users/lokinfey/workshop', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7974da432020>, 'serialize': <msrest.serialization.Serializer object at 0x7974d997f5b0>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/901e11c6-42ed-4fcc-875a-6bd77390af5b/resourceGroups/AzureAIGroup/workspaces/slm-workshop/datastores/worksp

In [ ]:
# Display model details
print(f"Model Name: {registered_model.name}")
print(f"Model ID: {registered_model.id}")
print(f"Model Version: {registered_model.version}")
print(f"Model Description: {registered_model.description}")
print(f"\nModel Location: {registered_model.path}")
print(f"\nFull Model Properties:")
for key, value in registered_model.__dict__.items():
    print(f"{key}: {value}")